In [2]:
import os
from bs4 import BeautifulSoup

In [42]:
slozka = "filmy"

In [43]:
def seznam_souboru(slozka):
    seznam = []
    for file in os.listdir(slozka):
        seznam.append(file)
    return(seznam)

In [44]:
seznam_person = seznam_souboru(slozka)
seznam_person[0:10]

['10144-tabor-padlych-zen.html',
 '10173-eastern-krvavy-hugo.html',
 '102282-o-jecminkovi.html',
 '102314-zimni-vila.html',
 '10242-narkoblues.html',
 '103341-jan-hus-mse-za-tri-mrtve-muze.html',
 '103354-obcansky-prukaz.html',
 '103358-7-dni-hrichu.html',
 '103360-vojta-lavicka-nahoru-a-dolu.html',
 '103369-kuky-se-vraci.html']

## Začínáme scrapovat

In [45]:
import pandas as pd

In [46]:
def id_filmu(url):
    if "cs/film/" in url:
        id_filmu = url.split("/")[-2]
    else:
        id_filmu = 0
    return(id_filmu)

In [56]:
def id_persony(url):
    if "cs/person/" in url:
        id_persony = url.split("/")[-2]
    else:
        id_persony = 0
    return(id_persony)

In [143]:
def scrape_role(slozka, soubor):
    
    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
        
    item_details = soup.find_all(class_='db-film-credits-section')
    
    snimek = []
    
    for item_detail in item_details:
        
        if "Hrají" in item_detail.find("h2").text:
            
            divs = item_detail.find_all(class_="credit-item")

            for d in divs:
                
                role = {}
                
                role["Film"] = f"""{soup.find("h1").text.strip()} ({soup.find('meta', {'property': 'og:url'})['content'].split("/")[-2]})"""
    
                uloha = None
                hraje = None
                mluvi = None
                
                if d.find("h3").find("a"):
                    hraje = id_persony(d.find("h3").find("a")["href"])
    
                else:
                    hraje = d.find("h3").text.strip()
                    
                if d.find("br"):
                    try:
                        uloha = d.find("p").find("br").next_sibling.text.strip()
                    except:
                        pass
                    
                    if d.find("span").text.strip() == "Mluví":
                        if d.find("p").find("a"):
                            mluvi = id_persony(d.find("p").find("a")["href"])                        
                            
                else:
                    if d.find("p"):
                        uloha = d.find("p").text.strip()
                
                if uloha:
                    if "/" in uloha and uloha[-1] != "/":
                            uloha = uloha.split("/")
                            for u in uloha:
                                role["Role"] = u.strip()
                                role["Hraje"] = hraje
                                role["Mluví"] = mluvi
                    
                role["Role"] = uloha
                role["Hraje"] = hraje
                role["Mluví"] = mluvi

                snimek.append(role)
                
    return pd.DataFrame(snimek)

## Test

In [137]:
test = scrape_role("filmy","10173-eastern-krvavy-hugo.html")
test

{'Film': 'Eastern – Krvavý Hugo (10173)', 'Role': 'Krvavý Hugo', 'Hraje': 'Štefan Capko', 'Mluví': None}
{'Film': 'Eastern – Krvavý Hugo (10173)', 'Role': 'stařenka', 'Hraje': 'Hedvika Krupičková', 'Mluví': None}
{'Film': 'Eastern – Krvavý Hugo (10173)', 'Role': 'holčička', 'Hraje': 'Lucie Štepánková', 'Mluví': None}
{'Film': 'Eastern – Krvavý Hugo (10173)', 'Role': 'pradlenka', 'Hraje': 'Denisa Nová', 'Mluví': None}
{'Film': 'Eastern – Krvavý Hugo (10173)', 'Role': 'muž', 'Hraje': '3277', 'Mluví': None}
{'Film': 'Eastern – Krvavý Hugo (10173)', 'Role': 'žena', 'Hraje': 'Zuzana Veselá', 'Mluví': None}
{'Film': 'Eastern – Krvavý Hugo (10173)', 'Role': None, 'Hraje': 'David Heger', 'Mluví': None}
{'Film': 'Eastern – Krvavý Hugo (10173)', 'Role': None, 'Hraje': '3004', 'Mluví': None}
{'Film': 'Eastern – Krvavý Hugo (10173)', 'Role': 'komando', 'Hraje': '1189', 'Mluví': None}


,Film,Role,Hraje,Mluví
0,Eastern – Krvavý Hugo (10173),Krvavý Hugo,Štefan Capko,None
1,Eastern – Krvavý Hugo (10173),stařenka,Hedvika Krupičková,None
2,Eastern – Krvavý Hugo (10173),holčička,Lucie Štepánková,None
3,Eastern – Krvavý Hugo (10173),pradlenka,Denisa Nová,None
4,Eastern – Krvavý Hugo (10173),muž,3277,None
5,Eastern – Krvavý Hugo (10173),žena,Zuzana Veselá,None
6,Eastern – Krvavý Hugo (10173),None,David Heger,None
7,Eastern – Krvavý Hugo (10173),None,3004,None
8,Eastern – Krvavý Hugo (10173),komando,1189,None


In [144]:
test = scrape_role("filmy","402178-pisen-nemilovaneho.html")
test

,Film,Role,Hraje,Mluví
0,Píseň nemilovaného (402178),"básník Guillaume Apollinaire, vlastním jménem ...",111618,1674
1,Píseň nemilovaného (402178),"malířka Marie Laurencinová, přítelkyně Guillauma",Christine Boisson /ž/,1427
2,Píseň nemilovaného (402178),"učitelka Madeleine Pagèsová, snoubenka Guillauma",Béatrice Bruno /ž/,844
3,Píseň nemilovaného (402178),"Madame Angelica de Kostrowitzky, matka Guillauma",5950,None
4,Píseň nemilovaného (402178),"důstojník Francesco Flugi d’Aspermont, otčím G...",7967,None
...,...,...,...,...
113,Píseň nemilovaného (402178),"[hudebník v karnevalovém průvodu, hudebník v k...",Luboš Hadač,None
114,Píseň nemilovaného (402178),"[hudebník v karnevalovém průvodu, hudebník v k...",Jan Šmíd,None
115,Píseň nemilovaného (402178),"[hudebník v karnevalovém průvodu, hudebník v k...",Bohumil Holeček,None
116,Píseň nemilovaného (402178),"[hudebník v karnevalovém průvodu, hudebník v k...",Josef Pospíšil,None


## Scrapujeme herectvo

In [124]:
def scrape_herectvo(slozka):
    herectvo = pd.DataFrame()
    count = 0 
    for s in seznam_souboru(slozka)[:kolik]:
        count = count + 1
        print(f"done: {count} / total: {len(seznam_souboru(slozka)[:kolik])} / {s}")
        h = scrape_role(slozka, s)
        herectvo = pd.concat([herectvo, h])
    herectvo = herectvo.reset_index(drop=True)
    return(herectvo)

In [125]:
kolik = None

In [145]:
herectvo = scrape_herectvo("filmy")

done: 1 / total: 4562 / 10144-tabor-padlych-zen.html
done: 2 / total: 4562 / 10173-eastern-krvavy-hugo.html
done: 3 / total: 4562 / 102282-o-jecminkovi.html
done: 4 / total: 4562 / 102314-zimni-vila.html
done: 5 / total: 4562 / 10242-narkoblues.html
done: 6 / total: 4562 / 103341-jan-hus-mse-za-tri-mrtve-muze.html
done: 7 / total: 4562 / 103354-obcansky-prukaz.html
done: 8 / total: 4562 / 103358-7-dni-hrichu.html
done: 9 / total: 4562 / 103360-vojta-lavicka-nahoru-a-dolu.html
done: 10 / total: 4562 / 103369-kuky-se-vraci.html
done: 11 / total: 4562 / 103372-habermannuv-mlyn.html
done: 12 / total: 4562 / 103374-dobre-placena-prochazka.html
done: 13 / total: 4562 / 103402-nebe-peklo.html
done: 14 / total: 4562 / 103495-lidice.html
done: 15 / total: 4562 / 103499-medvedi-ostrovy.html
done: 16 / total: 4562 / 103509-dont-stop.html
done: 17 / total: 4562 / 104213-az-na-veky.html
done: 18 / total: 4562 / 10574-elfilm.html
done: 19 / total: 4562 / 106663-dum.html
done: 20 / total: 4562 / 1066

done: 157 / total: 4562 / 16421-ostrov-lasky.html
done: 158 / total: 4562 / 16423-waterloo-po-cesku.html
done: 159 / total: 4562 / 164346-certova-nevesta.html
done: 160 / total: 4562 / 16439-zacatek-sveta.html
done: 161 / total: 4562 / 16450-vrazda-septanym-slovem-na-velikonoce-o-velkem-patku.html
done: 162 / total: 4562 / 16453-zatmeni-nadeji-o-ctyci-mesice-pozdeji-11-8-1999.html
done: 163 / total: 4562 / 16454-panny-posetile-o-dalsi-ctyri-mesice-pozdeji-31-12-1999.html
done: 164 / total: 4562 / 164669-westernstory.html
done: 165 / total: 4562 / 165457-praho-ma-lasko.html
done: 166 / total: 4562 / 165508-bastardi.html
done: 167 / total: 4562 / 1659-mach-sebestova-a-kouzelne-sluchatko.html
done: 168 / total: 4562 / 1661-kralovsky-slib.html
done: 169 / total: 4562 / 166198-vendeta.html
done: 170 / total: 4562 / 166202-cigan.html
done: 171 / total: 4562 / 166279-vera-68.html
done: 172 / total: 4562 / 166281-nejvetsi-prani.html
done: 173 / total: 4562 / 167017-lezaky-42.html
done: 174 / t

done: 310 / total: 4562 / 22344-dvojrole.html
done: 311 / total: 4562 / 22394-eliska-ma-rada-divocinu.html
done: 312 / total: 4562 / 22433-ene-bene.html
done: 313 / total: 4562 / 22534-fontana-pro-zuzanu-3.html
done: 314 / total: 4562 / 22572-hanele.html
done: 315 / total: 4562 / 22875-non-plus-ultras.html
done: 316 / total: 4562 / 22888-horem-padem.html
done: 317 / total: 4562 / 22896-kamenak-2.html
done: 318 / total: 4562 / 22899-zralok-v-hlave.html
done: 319 / total: 4562 / 23443-cesky-sen.html
done: 320 / total: 4562 / 23638-choking-hazard.html
done: 321 / total: 4562 / 23639-vratne-lahve.html
done: 322 / total: 4562 / 23661-byl-jsem-mladistvym-intelektualem.html
done: 323 / total: 4562 / 23667-kanarek.html
done: 324 / total: 4562 / 23974-spravce-statku.html
done: 325 / total: 4562 / 23981-hrubes-a-mares-jsou-kamaradi-do-deste.html
done: 326 / total: 4562 / 24015-mir-jejich-dusi.html
done: 327 / total: 4562 / 24064-bolero.html
done: 328 / total: 4562 / 24194-lazebnik-sibirsky.html


done: 467 / total: 4562 / 395163-dobrodruzstvi-joe-focka.html
done: 468 / total: 4562 / 395164-ferenc-se-zeni.html
done: 469 / total: 4562 / 395165-kozlonoh.html
done: 470 / total: 4562 / 395166-laska-si-neda-poroucet.html
done: 471 / total: 4562 / 395167-lasko-trikrat-svata.html
done: 472 / total: 4562 / 395168-na-pomoc-dohode.html
done: 473 / total: 4562 / 395169-o-devcicu.html
done: 474 / total: 4562 / 395170-osalena-komtesa-zuzana.html
done: 475 / total: 4562 / 395171-praha-roku-1549.html
done: 476 / total: 4562 / 395172-princezna-z-chalupy.html
done: 477 / total: 4562 / 395173-sen-fratera-ondreje.html
done: 478 / total: 4562 / 395174-sestnactileta.html
done: 479 / total: 4562 / 395175-ucitel-orientalnich-jazyku.html
done: 480 / total: 4562 / 395176-v-mesici-lasky.html
done: 481 / total: 4562 / 395177-ada-se-uci-jezdit.html
done: 482 / total: 4562 / 395178-akcie-p-z-h.html
done: 483 / total: 4562 / 395179-akord-smrti.html
done: 484 / total: 4562 / 395180-aloisuv-los.html
done: 485 

done: 621 / total: 4562 / 395318-kdyz-srdce-promluvi.html
done: 622 / total: 4562 / 395319-madame-golvery.html
done: 623 / total: 4562 / 395320-muz-bez-srdce.html
done: 624 / total: 4562 / 395321-palicova-dcera.html
done: 625 / total: 4562 / 395322-pepanek-nezdara.html
done: 626 / total: 4562 / 395323-pro-a-proti.html
done: 627 / total: 4562 / 395324-problematicky-gentleman.html
done: 628 / total: 4562 / 395325-rasputin.html
done: 629 / total: 4562 / 395326-tu-ten-kamen.html
done: 630 / total: 4562 / 395327-unos-bankere-fuxe.html
done: 631 / total: 4562 / 395328-za-oponou-smrti.html
done: 632 / total: 4562 / 395329-zahadny-pripad-galginuv.html
done: 633 / total: 4562 / 395330-zavet-podivinova.html
done: 634 / total: 4562 / 395331-bily-raj.html
done: 635 / total: 4562 / 395332-chytte-ho.html
done: 636 / total: 4562 / 395333-dama-z-baru.html
done: 637 / total: 4562 / 395334-devce-z-hor.html
done: 638 / total: 4562 / 395335-dvoji-zivot.html
done: 639 / total: 4562 / 395336-hrichy-v-manzel

...

done: 3891 / total: 4562 / 401839-mimi-liza.html
done: 3892 / total: 4562 / 401840-kdyz-draka-boli-hlava.html
done: 3893 / total: 4562 / 401841-zlaty-podraz.html
done: 3894 / total: 4562 / 401842-ten-kdo-te-miloval.html
done: 3895 / total: 4562 / 401843-sklep.html
done: 3896 / total: 4562 / 401844-neklidna-hranice.html
done: 3897 / total: 4562 / 401845-po-cem-muzi-touzi.html
done: 3898 / total: 4562 / 401846-cesta-vede-do-tibetu.html
done: 3899 / total: 4562 / 401847-na-stojaka-v-kine.html
done: 3900 / total: 4562 / 401848-manzelske-etudy-nova-generace.html
done: 3901 / total: 4562 / 401851-iluze.html
done: 3902 / total: 4562 / 401852-univerzity-a-svoboda.html
done: 3903 / total: 4562 / 401853-zeny-v-behu.html
done: 3904 / total: 4562 / 401854-moje-stoleti.html
done: 3905 / total: 4562 / 401855-kapela.html
done: 3906 / total: 4562 / 401856-central-bus-station.html
done: 3907 / total: 4562 / 401857-cukr.html
done: 3908 / total: 4562 / 401858-narusitel.html
done: 3909 / total: 4562 / 401

done: 4036 / total: 4562 / 401988-konzultace-posledni-den-zkouskoveho-obdobi.html
done: 4037 / total: 4562 / 401989-spolu-sami.html
done: 4038 / total: 4562 / 401990-starici.html
done: 4039 / total: 4562 / 401991-prvni-akcni-hrdina-a-jako-bonus-prvni-akcni-dokument.html
done: 4040 / total: 4562 / 401992-jeste-nekoncime.html
done: 4041 / total: 4562 / 401993-narodni-trida.html
done: 4042 / total: 4562 / 401994-pan-ponrepo-se-klani.html
done: 4043 / total: 4562 / 401995-plac-svateho-sebestiana.html
done: 4044 / total: 4562 / 401996-abstinent.html
done: 4045 / total: 4562 / 401997-prvni-akcni-hrdina.html
done: 4046 / total: 4562 / 401998-prvni-akcni-dokument.html
done: 4047 / total: 4562 / 401999-v-zajeti-site.html
done: 4048 / total: 4562 / 402000-hra.html
done: 4049 / total: 4562 / 402001-posledni-aristokratka.html
done: 4050 / total: 4562 / 402002-tiche-doteky.html
done: 4051 / total: 4562 / 402003-prazske-orgie.html
done: 4052 / total: 4562 / 402004-uzly-a-pomerance.html
done: 4053 / 

done: 4185 / total: 4562 / 402147-gump-pes-ktery-naucil-lidi-zit.html
done: 4186 / total: 4562 / 402148-muj-otec-antonin-kratochvil.html
done: 4187 / total: 4562 / 402149-na-krev.html
done: 4188 / total: 4562 / 402150-tichy-spolecnik.html
done: 4189 / total: 4562 / 402151-budiz-voda.html
done: 4190 / total: 4562 / 402152-jedine-tereza.html
done: 4191 / total: 4562 / 402153-martanske-lode.html
done: 4192 / total: 4562 / 402154-atlas-ptaku.html
done: 4193 / total: 4562 / 402155-krystof.html
done: 4194 / total: 4562 / 402156-kurz-manzelske-touhy.html
done: 4195 / total: 4562 / 402157-okupace.html
done: 4196 / total: 4562 / 402158-shoky-morthy-posledni-velka-akce.html
done: 4197 / total: 4562 / 402159-spici-mesto.html
done: 4198 / total: 4562 / 402160-jak-jsem-se-stala-partyzankou.html
done: 4199 / total: 4562 / 402162-dovolena.html
done: 4200 / total: 4562 / 402163-postacka-pohadka.html
done: 4201 / total: 4562 / 402164-rene-vezen-svobody.html
done: 4202 / total: 4562 / 402165-sila.html
d

done: 4335 / total: 4562 / 53059-piaty-statocny-martin-repka-zlodeji.html
done: 4336 / total: 4562 / 53102-siesty-statocny-aurel-klimt-o-kouzelnem-zvonu.html
done: 4337 / total: 4562 / 53125-vsichni-moji-blizci.html
done: 4338 / total: 4562 / 53139-z-pekla-stesti.html
done: 4339 / total: 4562 / 53143-venkovsky-ucitel.html
done: 4340 / total: 4562 / 53335-klicek.html
done: 4341 / total: 4562 / 53571-cernobila-v-barve.html
done: 4342 / total: 4562 / 53944-kulicky.html
done: 4343 / total: 4562 / 53961-hannibal-zrozeni.html
done: 4344 / total: 4562 / 53971-tulaci.html
done: 4345 / total: 4562 / 54020-zlati-uhori.html
done: 4346 / total: 4562 / 54060-certe-drz-se-sveho-kopyta.html
done: 4347 / total: 4562 / 54067-malvina.html
done: 4348 / total: 4562 / 54076-pan-hradu.html
done: 4349 / total: 4562 / 54085-vohnice-a-kilijan.html
done: 4350 / total: 4562 / 54091-dame-kord-neslusi.html
done: 4351 / total: 4562 / 54224-vaclav.html
done: 4352 / total: 4562 / 54226-goyovy-prizraky.html
done: 4353

done: 4487 / total: 4562 / 8141-v-erbu-lvice.html
done: 4488 / total: 4562 / 8142-jak-si-zaslouzit-princeznu.html
done: 4489 / total: 4562 / 8249-hrad-z-pisku.html
done: 4490 / total: 4562 / 8254-saturnin.html
done: 4491 / total: 4562 / 8259-strelec.html
done: 4492 / total: 4562 / 82817-draci.html
done: 4493 / total: 4562 / 8296-indianske-leto.html
done: 4494 / total: 4562 / 8404-valka-barev.html
done: 4495 / total: 4562 / 8405-divoke-pivo.html
done: 4496 / total: 4562 / 8461-fany.html
done: 4497 / total: 4562 / 8462-jak-chutna-smrt.html
done: 4498 / total: 4562 / 8480-kone-na-betone.html
done: 4499 / total: 4562 / 8531-kouzelny-mesec.html
done: 4500 / total: 4562 / 8552-ani-smrt-nebere.html
done: 4501 / total: 4562 / 856-kruh.html
done: 4502 / total: 4562 / 8578-horke-leto.html
done: 4503 / total: 4562 / 87180-jan-69.html
done: 4504 / total: 4562 / 8749-uz.html
done: 4505 / total: 4562 / 8785-buldocek.html
done: 4506 / total: 4562 / 8786-posledni-stisko-pana-zorze.html
done: 4507 / to

## Export

In [146]:
import os

In [147]:
herectvo.to_json(os.path.join("data","herectvo_raw.json"),orient="records")